In [ ]:

%pip install requests
%pip install pandas
%pip install boto3
%pip install awswrangler

In [2]:
import requests
import pandas as pd
import boto3
import awswrangler as wr



In [27]:
#defining a function to retrieve a stored API KEY from AWS System manager
def get_parameter(parameter_name, region):
    client = boto3.client('ssm', region_name=region)
    parameter_response = client.get_parameter(Name=parameter_name)
    return parameter_response["Parameter"]["Value"]

In [ ]:
api_key = get_parameter('bimmy_API_key','eu-central-1')
api_key

In [ ]:
def GuardianAPI(base_url, total_pages:range, search_keyword:str, date_range, my_key: str,):
    """
    This function makes a call to guardian API based on the number of pages specified.
    It saves the articles into an empty list, which is later converted into a dataframe.
    
    Arguments: 
        base_url: valid base_url from the guardian documentation: https://open-platform.theguardian.com/documentation/
        total_pages: a valid object range, e.g range(1,4)
        search_keyword: the keywords to search as part of the call
        date_range: date range - check the Guardian documentation on how to specify the date range
        api_key: a valid personal API key """
    full_movies = []
    for page in total_pages:
        full_endpoint = f"{base_url}?page={page}&q={search_keyword}&{date_range}&api-key={api_key}"
        output = requests.get(url = full_endpoint)
        if output.status_code != 200:
            raise Exception(f"the dataset is not found due to status code is {output.status_code}")
            for entries in raw_movies.json()['response']['results']:
                full_movies.append(entries)
    movies_dataset = pd.DataFrame.from_records(full_movies)
    return movies_dataset

In [ ]:
GuardianAPI(
    "https://content.guardianapis.com",
    range(1,586),
    "movies",
    "from-date=2020-04-16&to-date=2024-04-16"
    get_parameter('bimmy_API_key','eu-central-1'))

In [ ]:
#storing dataset into s3 bucket in AWS 
def movies_to_load(destination, dataframe_name):
    wr.s3.to_parquet(
        df=dataframe_name,
        path=destination
        dataset=True)
    
movies_to_load(destination="s3://bimmy-module-bucket/GuardianMovies.parquet/",
               dataframe_name=movies_dataset)